<a href="https://colab.research.google.com/github/ChaithanyaSaiB/UMBC-DATA606-Capstone/blob/main/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [5]:
cnnhealth = pd.read_csv("/content/cnnhealth.txt", sep=r"(?<!\s)[|](?!\s)", names=['ID', 'DateTime', 'Article Title and Link'], engine="python")
cnnhealth.head()

,ID,DateTime,Article Title and Link
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [6]:
cnnhealth['URL'] = [re.findall(r'https?://\S+', text) for text in cnnhealth['Article Title and Link']]
cnnhealth.head(2)

,ID,DateTime,Article Title and Link,URL
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,[http://cnn.it/1L1t1Fv]
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...,"[http://cnn.it/1xdpsjT, http://pbs.twimg.com/m..."


In [7]:
cnnhealth['URL'].apply(len).value_counts()

1    3298
0     456
2     305
Name: URL, dtype: int64

In [8]:
drop_indices = cnnhealth['Article Title and Link'][cnnhealth['URL'].apply(len) != 1].index
cnnhealth.drop(drop_indices, inplace=True)
cnnhealth['URL'] = [url[0] for url in cnnhealth['URL']]
cnnhealth.head()

,ID,DateTime,Article Title and Link,URL
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk
9,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg
17,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.
20,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC


In [9]:
cnnhealth_subset = cnnhealth[:5].copy()

In [15]:
import pandas as pd

# Function to fetch the content from a URL
def fetch_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print("Error fetching content:", e)
        return None

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content):
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        article_body = soup.findAll('p', class_='paragraph')
        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
cnnhealth_subset['Content'] = cnnhealth_subset['URL'].apply(lambda url: extract_body(fetch_content(url)))

# Print the DataFrame with the extracted content
cnnhealth_subset.head()


,ID,DateTime,Article Title and Link,URL,Content
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv,"[[\n When I was a child, a pale spe..."
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk,[[\n “Hey dude – can you turn your ...
9,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg,[[\n The misuse of outer protective...
17,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.,[[\n The National Institutes of Hea...
20,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC,[[\n Robert Downey Jr. may be Iron ...


In [16]:
cnnhealth_subset['Content'][0]

[<p class="paragraph inline-placeholder" data-article-gutter="true" data-component-name="paragraph" data-editable="text" data-uri="archive.cms.cnn.com/_components/paragraph/instances/paragraph_83B33ADE-F3C6-6376-842A-025BAB2016D6@published">
             When I was a child, a pale specter used to call our house most evenings, eager to chat with my doctor father about her myriad medical concerns. 
     </p>,
 <p class="paragraph inline-placeholder" data-article-gutter="true" data-component-name="paragraph" data-editable="text" data-uri="archive.cms.cnn.com/_components/paragraph/instances/paragraph_06C7002A-FEFC-090F-5196-024502D1874C@published">
             We called her the “White Bread Lady,” a moniker she earned for one particularly inane call in which she panicked to my father after consuming white bread. 
     </p>,
 <p class="paragraph inline-placeholder" data-article-gutter="true" data-component-name="paragraph" data-editable="text" data-uri="archive.cms.cnn.com/_components/para